<a href="https://colab.research.google.com/github/HAL22/Kaggle-Competitions/blob/contradictory/Contradictory_My_Dear_Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
%%capture
!pip install kaggle
!pip install -q googletrans==3.1.0a0
!pip install datasets transformers[sentencepiece]
!apt install git-lfs
!pip install tensorflow-addons==0.16.1
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from transformers import AutoTokenizer
import pandas as pd
from datasets import load_dataset
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForSequenceClassification
from sklearn.model_selection import StratifiedKFold
from transformers import TFXLMRobertaModel
from googletrans import Translator
from dask import bag, diagnostics # extends common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments
from huggingface_hub import notebook_login
from tensorflow_addons.optimizers import RectifiedAdam
from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!git config --global user.email "thethelafaltein@gmail.com"
!git config --global user.name "HAL22"

In [4]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
%%capture
!unzip '/content/test.csv.zip'
!unzip '/content/train.csv.zip'

In [6]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

In [7]:
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


**Translation**

In [8]:
translations_csv = '/content/train_translations.csv'

dest_choices = {'ar': 'Arabic', 'fr': 'French', 'sw': 'Swahili', 'ur': 'Urdu', 'vi': 'Vietnamese', 'ru': 'Russian', 'hi': 'Hindi', 'el': 'Greek', 'th': 'Thai', 'es': 'Spanish', 'de': 'German', 'tr': 'Turkish', 'bg': 'Bulgarian'}

In [9]:
def translate(words, dest):
            translator = Translator()
            decoded = translator.translate(words, dest=dest).text
            return decoded

In [10]:
# Example
translate("How are you","fr")

'Comment allez-vous'

In [11]:
def translate_job(df, mode='en'):
   premise_bag = bag.from_sequence(df.premise.tolist())
   hypothesis_bag = bag.from_sequence(df.hypothesis.tolist())

   if mode == 'en':
     df['translate_to_abv'] = 'en'
     df['translate_to'] = 'English'
     dest_bag = bag.from_sequence(df.translate_to_abv.tolist())
   elif mode == 'random':
     df['translate_to_abv'] = np.random.choice(list(dest_choices.keys()), df.shape[0])
     df['translate_to'] = df.translate_to_abv.map(dest_choices)
     dest_bag = bag.from_sequence(df.translate_to_abv.tolist())

   with diagnostics.ProgressBar():
     premise_translated = premise_bag.map(translate, dest=dest_bag).compute()
     hypothesis_translated = hypothesis_bag.map(translate, dest=dest_bag).compute()

   df['premise'] = premise_translated
   df['hypothesis'] = hypothesis_translated
   df['id'] = df['id'].astype(str) + '_' + df['translate_to_abv']
   df['lang_abv'] = df['translate_to_abv']
   df['language'] = df['translate_to']
   df['augmented'] = True
   df.drop('translate_to_abv', axis=1, inplace=True)
   df.drop('translate_to', axis=1, inplace=True)

   return df   

In [12]:
eng = df_train.loc[df_train.lang_abv == "en"].copy().pipe(translate_job, mode='random')

[########################################] | 100% Completed |  7min 36.2s
[########################################] | 100% Completed |  7min  2.9s


In [13]:
eng

,id,premise,hypothesis,lang_abv,language,label,augmented
0,5130fd2cb5_de,und diese Kommentare wurden bei der Formulieru...,Die zwischenzeitlich entwickelten Regeln wurde...,de,German,0,True
1,5b72532a0b_bg,"Това са проблеми, с които се борим в практичес...",Практическите групи нямат право да работят по ...,bg,Bulgarian,2,True
3,5622f0c60b_hi,आप जानते हैं कि वे वास्तव में खुद का बचाव नहीं...,वे अपनी उम्र के कारण अपना बचाव नहीं कर सकते।,hi,Hindi,0,True
7,fdcd1bd867_hi,कॉकपिट कंट्री से सेंट एन्स बे तक,सेंट एन्स बे से कॉकपिट कंट्री तक।,hi,Hindi,2,True
8,7cfb3d272c_ar,انظر ، إنها بشرتك ، لكنك ستكون في مشكلة إذا لم...,سيطردك الرئيس إذا رآك تتراخى.,ar,Arabic,1,True
...,...,...,...,...,...,...,...
12115,2b78e2a914_es,Los resultados de incluso los estudios epidemi...,Todos los estudios tienen la misma cantidad de...,es,Spanish,2,True
12116,7e9943d152_vi,Nhưng có hai loại niềm vui khi làm và niềm vui...,Nhưng có hai loại niềm vui khi làm và niềm vui...,vi,Vietnamese,0,True
12117,5085923e6c_vi,Điều quan trọng là nhận ra rằng đã quá lâu để ...,"Nó không thể được di chuyển, bây giờ hoặc mãi ...",vi,Vietnamese,2,True
12118,fc8e2fd1fe_es,En el extremo oeste hay un modelo detallado de...,El complejo del templo modelo está en el extre...,es,Spanish,2,True


In [14]:
to_eng = df_train.loc[df_train.lang_abv != "en"].copy().pipe(translate_job, mode='en')

[########################################] | 100% Completed |  6min 33.4s
[########################################] | 100% Completed |  5min 35.6s


In [15]:
to_eng

,id,premise,hypothesis,lang_abv,language,label,augmented
2,3931fbe82a_en,Little things like that make a huge difference...,I was trying to accomplish something.,en,English,0,True
4,86aaa48b45_en,In role play as well. Opportunities to express...,Children can see how different ethnicities are.,en,English,1,True
5,ed7d6a1e62_en,You might hear someone on a farm say we have t...,"On the farm, people use different terms.",en,English,0,True
6,5a0f4908a0_en,"Upon his return to the United States, Hajj was...",Hague was investigated by FBI agents,en,English,0,True
9,8c10229663_en,"Every hundred degrees, the spots of paint chan...",The paint changes according to the color.,en,English,0,True
...,...,...,...,...,...,...,...
12104,eae412f48e_en,"Yes, - said Ogle, - it's true. But there were ...",Ogle never mentioned opponents of the current ...,en,English,2,True
12106,31f0c3d2d3_en,C-R functions can also be estimated with or wi...,The C-R functions can be estimated in differen...,en,English,1,True
12111,39ea2c1e23_en,"-design, draw and hand sew all those gorgeous ...",The costumes were worked only by human hands.,en,English,1,True
12112,30a3361a15_en,We didn't know what U2 was and nobody knew any...,We did not know the name of what we saw in the...,en,English,1,True


In [16]:
df_train[['premise', 'hypothesis']].values.tolist()

[['and these comments were considered in formulating the interim rules.',
  'The rules developed in the interim were put together with these comments in mind.'],
 ['These are issues that we wrestle with in practice groups of law firms, she said. ',
  'Practice groups are not permitted to work on these issues.'],
 ["Des petites choses comme celles-là font une différence énorme dans ce que j'essaye de faire.",
  "J'essayais d'accomplir quelque chose."],
 ["you know they can't really defend themselves like somebody grown uh say my age you know yeah",
  "They can't defend themselves because of their age."],
 ['ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสดงออกและได้เล่นหลายบทบาทไปพร้อมกัน ๆ อาจช่วยให้เด็กจับความคล้ายคลึงและความแตกต่างระหว่างผู้คนในด้านความปรารถนา ความเชื่อ และความรู้สึกได้',
  'เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร'],
 ['Bir çiftlikte birisinin, ağıla kapatılmış bu öküzleri kesmeliyiz dediğini duyabilirsiniz bu muhtemelen şu anlama gelir, yüklenecek olanları ayır

In [17]:
df_train['augmented'] = False
df_train = df_train.append([eng])
df_train = df_train.append([to_eng])
df_train.reset_index(drop=True, inplace=True)
df_train.to_csv('/content/train_translations.csv', index=False)

In [18]:
df_train.head()

,id,premise,hypothesis,lang_abv,language,label,augmented
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,False
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,False
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,False
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,False
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,False


In [19]:
def encode(df, tokenizer, max_length=96):
    values = df[['premise', 'hypothesis']].values.tolist()
    encodings = tokenizer.batch_encode_plus(values, pad_to_max_length=True, max_length=max_length)
    return np.array(encodings['input_ids'])

In [20]:
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-xlm-roberta-large")

In [21]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(skf.split(df_train, df_train['label'])):
    df_train.loc[val_idx, 'fold'] = fold

In [26]:
def build_dataset(df, tokenizer, batch_size=16, label=True, 
                  shuffle=True, repeat=True, cache=True, augment=False,
                  max_length=96):
  
  if not augment:
        df = df.loc[df.augmented == False].copy()


    
        if label:

          ds = tf.data.Dataset.from_tensor_slices((encode(df, tokenizer, max_length), df['label'].values))
        else:

          ds = tf.data.Dataset.from_tensor_slices(encode(df, tokenizer, max_length))

        if shuffle:
          ds = ds.shuffle(batch_size, reshuffle_each_iteration=True)
        if repeat:
          ds = ds.repeat()
        if cache:
          ds = ds.cache()

        ds = ds.batch(batch_size, drop_remainder=shuffle)
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)

        return ds

AttributeError: ignored

In [55]:
def build_model(loss='sparse_categorical_crossentropy', dropout=0.30):


  roberta_model = TFXLMRobertaModel.from_pretrained("jplu/tf-xlm-roberta-base")
  input_ids = tf.keras.layers.Input(shape=(96,), dtype=tf.int32, name="input_ids")
  sequence_output = roberta_model(input_ids)[0]
  cls_token = sequence_output[:, 0, :]
  cls_token = tf.keras.layers.Dropout(dropout)(cls_token)
  cls_token = tf.keras.layers.Dense(32, activation='relu')(cls_token)
  cls_token = tf.keras.layers.Dense(16, activation='relu')(cls_token)
  out = tf.keras.layers.Dense(3, activation='softmax')(cls_token)
  model = tf.keras.Model(inputs=input_ids, outputs=out)

  optimizer = RectifiedAdam()

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  return model

In [56]:
model = build_model()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 96)]              0         
                                                                 
 tfxlm_roberta_model_3 (TFXL  TFBaseModelOutputWithPoo  278043648
 MRobertaModel)              lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             96, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [40]:
def lr_scheduler(batch_size=16):
    
    lr_start   = 0.00003
    lr_max     = 0.0000003 * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 2
    lr_sus_ep  = 0
    lr_decay   = 0.8
    
    def lr_fn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        return lr
    
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_fn, verbose=False)
    return lr_callback

_ = lr_scheduler()

In [41]:
def get_callbacks(fold, batch_size=16, patience=3):
    model_name = "jplu/tf-xlm-roberta-base".split('/')[1]
    checkpoint_path = f'/kaggle/working/{model_name}-fold-{fold}-weights.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', 
                                                    save_freq='epoch', mode='min', 
                                                    save_best_only=True, save_weights_only=True, 
                                                    verbose=1)
    early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=patience)
    return [checkpoint, early, lr_scheduler(batch_size)]

In [58]:

results = {}

for fold in range(3):
    
    if 'GPU' == 'TPU':
        tf.tpu.experimental.initialize_tpu_system(tpu)
    
    print(f'\n- Fold [{fold}] as validation set, and folds {[i for i in range(3) if i != fold]} as training set\n')

    train = df_train.loc[df_train.fold != fold].copy()
    val = df_train.loc[df_train.fold == fold].copy()

    train_dataset = build_dataset(train, 
                                  tokenizer,
                                  label=True, shuffle=True, 
                                  repeat=False, cache=True, 
                                  augment=True)

    val_dataset = build_dataset(val, 
                                tokenizer,
                                label=True, shuffle=False, 
                                repeat=False, cache=True,
                                augment=False)

    model = build_model()

    history = model.fit(np.array(train_dataset), 
                        epochs=3, 
                        batch_size=16, 
                        callbacks=get_callbacks(fold), 
                        validation_data=val_dataset,
                        steps_per_epoch=len(train) // 16)

    print('- Performing out-of-fold predictions...')
    print('- Loading best model...')
    model_name = "jplu/tf-xlm-roberta-base".split('/')[1]
    model.load_weights(f'/kaggle/working/{"jplu/tf-xlm-roberta-base"}-fold-{fold}-weights.h5')

    print('- Predicting on OFF validation set...')
    off_dataset = build_dataset(val,
                                tokenizer,
                                label=False, shuffle=False,
                                repeat=False, cache=True,
                                augment=True)

    prob = model.predict(off_dataset, verbose=1)
    val['pred'] = np.argmax(prob, axis=1)

    print('- Compute metrics...')
    auc = roc_auc_score(val['label'], prob, multi_class='ovr')
    accuracy = accuracy_score(val['label'], val['pred'])

    results[fold] = {'auc': auc, 'accuracy': accuracy}
    print(f'- AUC: {auc}, Accuracy: {accuracy}\n')

    del model
    gc.collect()
    tf.keras.backend.clear_session()

    experiment.end()


- Fold [0] as validation set, and folds [1, 2] as training set



/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


ValueError: ignored

In [ ]:
model.fit()

In [37]:
df_train = df_train[["premise", "hypothesis","label"]].copy()

In [38]:
train = Dataset.from_pandas(df_train)
train

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 48480
})

In [39]:
train[111]

{'premise': 'Nếu ai có ấn bản năm 1984, anh ta có thể sẽ phẫn nộ vì phải mua cuốn đó chứ không phải là một bản nhỏ hơn (và ít tốn kém hơn).',
 'hypothesis': 'Phụ phí rẻ hơn sách.',
 'label': 0}

In [40]:
tokenized_ds = train.map(tokenize_dataset, batched=True)

  0%|          | 0/49 [00:00<?, ?ba/s]

In [41]:
dsd = tokenized_ds.train_test_split(.25, seed=42)
dsd

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 36360
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 12120
    })
})

In [14]:
# For GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [44]:
model = AutoModelForSequenceClassification.from_pretrained(
    "jplu/tf-xlm-roberta-large", 
    hidden_dropout_prob=0.35, # The dropout probability for all fully connected layers in the embeddings, encoder, and pooler.
    attention_probs_dropout_prob=0.35, # The dropout ratio for the attention probabilities.
    num_labels=3,
).to(device) # The model uses cpu or gpu for training

OSError: ignored

KeyError: ignored

In [ ]:
tokenized_ds = train.map(tokenize_dataset, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
# Transformers assumes that labels have the column name "labels"
tokenized_ds = tokenized_ds.rename_columns({"label": "labels"})

In [ ]:
dsd = tokenized_ds.train_test_split(.25, seed=42)
dsd

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9090
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3030
    })
})

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels=pred.label_ids
    preds=pred.predictions.argmax(-1)
    f1=f1_score(labels, preds, average='weighted')
    ac=accuracy_score(labels, preds)
    return {"accuracy":ac, "f1":f1}

In [ ]:
from transformers import TrainingArguments, Trainer

bs = 16
epochs = 1
lr = 4e-5

In [ ]:
args = TrainingArguments(
    output_dir="./results",
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8, 
    weight_decay=0.01,)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=dsd['train'], 
    eval_dataset=dsd['test'],
    tokenizer=tokenizer, 
    compute_metrics=compute_metrics)

In [ ]:

torch.cuda.empty_cache()

In [ ]:
trainer.train();

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.492500
1000,0.623000
1500,0.888900
2000,0.849000
2500,0.780900
3000,0.730100
3500,0.694600
4000,0.640800
4500,0.611100
